In [1]:
import pandas as pd 
import json
import os
from config import CLIENT_ID, CLIENT_SECRET
import requests
import foursquare
import time

# Load JSON
filepath = os.path.join("Resources", "geojson.json")
with open(filepath) as jsonfile:
    geo_json = json.load(jsonfile)
nycdata = geo_json["features"]
nycdata[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [2]:
column_names = ['Borough','Neighborhood', 'Latitude', 'Longitude']
neighborhoods = pd.DataFrame(columns=column_names)

for data in nycdata:
    borough = nycdata = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    
    neighborhoodlatlon = data['geometry']['coordinates']
    lat = neighborhoodlatlon[1]
    lng = neighborhoodlatlon[0]
    
    neighborhoods = neighborhoods.append({'Borough':borough,
                                          'Neighborhood':neighborhood_name,
                                          'Latitude': lat,
                                          'Longitude': lng}, ignore_index=True)

neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [3]:
neighborhoods = neighborhoods.loc[neighborhoods["Borough"] == "Manhattan", :]

In [4]:
VERSION = 20180405
lat = 40.7484
lon = -73.9857
radius = 500 #Meters
categoryId = '4bf58dd8d48988d1fe941735'
limit = 1
endpoint = 'https://api.foursquare.com/v2/venues/search?'

url = '{}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
    endpoint,
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    lon,
    radius,
    categoryId,
    limit
)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cffdf3e4c1f6753b7f8f4ea'},
 'response': {'venues': [{'id': '50364f25e4b0aae9b3420bb1',
    'name': 'J. Landress Brass',
    'contact': {},
    'location': {'address': '153 W 36th St Ste 6',
     'crossStreet': '7th Avenue and Broadway',
     'lat': 40.75208860835749,
     'lng': -73.98900311784871,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.75208860835749,
       'lng': -73.98900311784871}],
     'distance': 496,
     'postalCode': '10018',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['153 W 36th St Ste 6 (7th Avenue and Broadway)',
      'New York, NY 10018',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fe941735',
      'name': 'Music Store',
      'pluralName': 'Music Stores',
      'shortName': 'Music Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/music_instruments_',
       'suffix': '.png'},
    

In [5]:
def create_url(endpoint, CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, categoryId):

    url = '{}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
        endpoint,
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,
        categoryId,
        limit
    )
    return url

endpoint = 'https://api.foursquare.com/v2/venues/search?'
categoryIds = ['4bf58dd8d48988d1e5931735',# Music Venue
               '4bf58dd8d48988d1e7931735',# Jazz Club
               '4bf58dd8d48988d1e8931735',# Piano Bar
               '4bf58dd8d48988d1e9931735',# Rock Club
               '5267e4d9e4b0ec79466e48d1',# Music Festival
               '56aa371be4b08b9a8d5734db',# Amphitheater
               '5032792091d4c4b30a586d5c',# Concert Hall
               '4d4b7105d754a06376d81259',# Nightlife Spot
               '4bf58dd8d48988d120941735',# Kareoke Bar
               ]
VERSION = 20180405
limit = 50
radius = 1000
lat = 45.0000000
lng = -70.0000000
categoryId = ','.join(categoryIds)
url_example = create_url(endpoint, CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, categoryId)
print(url_example)

# client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
# for venue in venues:
#     client.venues(venue)
#     client.venues.search(params={'query': 'coffee', 'll': '40.7233,-74.0030'})


https://api.foursquare.com/v2/venues/search?&client_id=ZZOWEUFEZ2LJEQLVHJ3LSFP40KSRDO4INKP5YHA244055N2L&client_secret=FGIZTT1ZG4IZU4NZBMXS1ZTHSBFDKXQJS4EQG0Z2K2QR5OXH&v=20180405&ll=45.0,-70.0&radius=1000&categoryId=4bf58dd8d48988d1e5931735,4bf58dd8d48988d1e7931735,4bf58dd8d48988d1e8931735,4bf58dd8d48988d1e9931735,5267e4d9e4b0ec79466e48d1,56aa371be4b08b9a8d5734db,5032792091d4c4b30a586d5c,4d4b7105d754a06376d81259,4bf58dd8d48988d120941735&limit=50


In [ ]:
def getNearby(neighborhoods, latitudes, longitudes, radius=1000):
    endpoint = endpoint = 'https://api.foursquare.com/v2/venues/search?'
    venues_list = []
    
    for hood_name, lat, lng, in zip(neighborhoods, latitudes, longitudes):
        url = create_url(endpoint, CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, categoryId)
        
        results = requests.get(url).json()['response']['venues']
        
        for item in results:
            venue_name = item['name']
            venue_id = item['id']
            venue_category = item['categories'][0]['name']
            venue_lat = item['location']['lat']
            venue_lng = item['location']['lng']
            category_id = item['categories'][0]['id']
            try:
                venue_city = item['location']['city']
            except:
                venue_city = 'N/A'
            venue_state = item['location']['state']
            
            venues_list.append([(hood_name,
                                lat,
                                lng,
                                venue_name,
                                venue_id,
                                category_id,
                                venue_category,
                                venue_lat,
                                venue_lng,
                                venue_city,
                                venue_state
                                )])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue Name',
                             'Venue ID',
                             'Category ID',
                             'Venue Category',
                             'Venue Latitude',
                             'Venue Longitude', 
                             'Venue City', 
                             'Venue State'
                            ]
    return nearby_venues

prelim_venue_data = getNearby(neighborhoods['Neighborhood'],
                              neighborhoods['Latitude'],
                              neighborhoods['Longitude'])

In [ ]:
prelim_venue_data.to_csv("output/summary_table.csv", index=False)

In [ ]:
prelim_venue_data["Category ID"].value_counts()

In [ ]:
newyork1 = newyork[0:449]
newyork2 = newyork[450:899]
newyork3 = newyork[900:1349]
newyork4 = newyork[1350:1799]
newyork5 = newyork[1800:2249]
newyork6 = newyork[2250:2514]

In [ ]:
newyork = prelim_venue_data[prelim_venue_data['Venue City']=='New York']
newyork.to_csv('output/newyork.csv', index=False)

In [ ]:
venues1 = newyork1['Venue ID']

def get_metrics(venues):
    client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, version = '20180405')
    metrics = []
    
    batchcounter = 1
    recordCount = 1
    for venue in venues:
        if recordCount%499 == 0:
            time.sleep(3601)
            batchcounter += 1
        
        else:
            newdata = client.venues(venue)
            for item in newdata:
                likeCount = newdata['venue']['likes']['count']
                try: 
                    ratings = newdata['venue']['rating']
                except: 
                    ratings = "None"
                try:
                    ratingSignals = newdata['venue']['ratingSignals']
                except:
                    ratingSignals = "None"

            metrics.append([venue,likeCount,ratings,ratingSignals])
            print(f"Collecting number {recordCount} of batch {batchcounter}")
            recordCount += 1

    venue_metrics = pd.DataFrame(metrics)
    venue_metrics.columns = ['Venue_ID',
                             'Likes',
                             'Rating',
                             'Rating Signals',
                            ]
    
    venue_metrics.to_csv('output/venue_metrics.csv',index = False)
    return venue_metrics
    
get_metrics(venues1)

In [ ]:
complete_data_set = pd.merge(prelim_data_set, venue_metrics, on="Venue ID", how='outer')